In [1]:
import numpy as np 
import math
import pickle
from scipy.sparse import coo_matrix
import networkx as nx
import random
import matplotlib.pyplot as plt
plt.style.use('default')

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit.primitives import Sampler, BackendSampler
from qiskit import Aer, QuantumRegister
from qiskit.circuit import QuantumCircuit

from os.path import dirname, abspath
import sys
parent_dir = dirname(dirname(abspath(' ')))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from src.optimizer.Quantum_Optimizer_FGSR_b import *
from src.bitflip_strategy.Local_Search_PUBO import local_search, comp_objective
from Cplex_HuboSolver import CPLEX_HUBOsolver

import warnings
warnings.simplefilter('ignore')

In [2]:
N = 127

In [3]:
with open('ibm_washington_5_hubo.pickle', mode='br') as fi:
  data = pickle.load(fi)

pubo_terms = {}
offset = 0

for key in data.keys():
    if key == -1:
        offset = data[key]
    else:
        pubo_terms[key] = data[key]

obj_data = [N, pubo_terms]

In [4]:
cplex_result = CPLEX_HUBOsolver().solve(obj_data)

refval = cplex_result.fval + offset

print(f'reference value: {refval:.3f}')

reference value: -198.000


In [5]:
ord = 3

pubo_data = []

for key in data.keys():
    if key == -1:
        pubo_offset = data[key]
    else:
        l = list(key)
        i = ord - len(l)
        if i > 0:
            for _ in range(i):
                l.append(l[-1])
        l.append(data[key])
        pubo_data.append(l)

pubo_data = np.asarray(pubo_data, dtype=float)
print(pubo_data)

[[  1.   1.   1.   4.]
 [  3.   3.   3.  -4.]
 [  5.   5.   5.   0.]
 ...
 [122. 123. 124.  -8.]
 [124. 126. 126.  -4.]
 [124. 125. 126.   8.]]


In [6]:
num_qubits = N

print(f'number of system qubits: {num_qubits}')

number of system qubits: 127


In [7]:
objective = lambda x: comp_objective(x, ord, pubo_data, pubo_offset)

In [8]:
# max number of random flips for each bitstring
n_flips = 50

def search(x):
    x = local_search(x, ord, pubo_data, pubo_offset, n_flips)
    return np.asarray(x, dtype=int)

In [9]:
n_reps = 2

q = QuantumRegister(num_qubits, name='q')
sub_qc = QuantumCircuit(q)
qc_params = ParameterVector(name='theta', length=num_qubits*n_reps)

for l in range(n_reps):
    k = 0
    while 2*k < num_qubits-1:
        sub_qc.cz(q[2*k], q[2*k+1])
        k += 1
            
    k = 0
    while 2*k+1 < num_qubits-1:
        sub_qc.cz(q[2*k+1], q[2*k+2])
        k += 1

    for k in range(num_qubits):
        sub_qc.ry(qc_params[l*num_qubits+k], q[k])

    sub_qc.barrier()

In [10]:
shots = 2**10
backend = Aer.get_backend('aer_simulator_matrix_product_state')

sampler = BackendSampler(backend=backend)
sampler.set_options(shots=shots)

print(f'shots = {shots}')

shots = 1024


In [11]:
# configure
alpha = 0.4
max_epoch = 1
restart_s = 0.8
restart_std = 0

In [12]:
QuantumSolver = QuantumOptimizer()
QuantumSolver.set_optimizer(num_qubits, sub_qc, sampler)

QuantumSolver.run(
    objective = objective,
    search = search,
    alpha = alpha,
    random_update = True,
    max_epoch = max_epoch,
    restart_s = restart_s,
    restart_std = restart_std,
    refval = refval,
    intermediate_result_display = True
    )

epoch = 0 	 step = 0 	 objval = -166.000 	 cvar = -19.415   	 s-prob = 0.000e+00
epoch = 0 	 step = 0 	 objval = -192.000 	 cvar = -159.507   	 s-prob = 0.000e+00
epoch = 0 	 step = 0 	 objval = -196.000 	 cvar = -179.122   	 s-prob = 0.000e+00
epoch = 0 	 step = 0 	 objval = -196.000 	 cvar = -178.141   	 s-prob = 0.000e+00
epoch = 0 	 step = 0 	 objval = -198.000 	 cvar = -181.429   	 s-prob = 2.930e-03
epoch = 0 	 step = 1 	 objval = -198.000 	 cvar = -182.278   	 s-prob = 3.906e-03
epoch = 0 	 step = 2 	 objval = -198.000 	 cvar = -181.454   	 s-prob = 3.906e-03
epoch = 0 	 step = 3 	 objval = -198.000 	 cvar = -182.673   	 s-prob = 4.883e-03
epoch = 0 	 step = 4 	 objval = -198.000 	 cvar = -182.610   	 s-prob = 2.930e-03
epoch = 0 	 step = 5 	 objval = -198.000 	 cvar = -181.907   	 s-prob = 1.074e-02
epoch = 0 	 step = 6 	 objval = -198.000 	 cvar = -182.185   	 s-prob = 7.812e-03
epoch = 0 	 step = 7 	 objval = -198.000 	 cvar = -182.244   	 s-prob = 3.906e-03
epoch = 0 	 step 

In [13]:
result = QuantumSolver.elog

L1 = [3*i+1 for i in range(len(result))]
L2 = list(result)
L3 = list(QuantumSolver.plog)
L4 = list(QuantumSolver.log)

In [14]:
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 14

plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['mathtext.default'] = 'it'

fig, ax = plt.subplots(figsize=(10, 4))
ax2 = ax.twinx()

ax.plot(L1, L2, color='navy', linewidth=1, label='Mean (Cost)')
ax.plot(L1, L4, color='darkgreen', linewidth=1, label='Best Sampled Value')

ax.set_ylabel('Cost (CVaR)')
ax.set_xlabel('Eval Count')

ax.hlines(refval, xmin=0, xmax=0, color='darkorange', label='Mean (Probability)')
ax.hlines(refval, xmin=1, xmax=L1[-1], color='grey', linestyle='dashed', linewidth=1, label='Optimal Value')
ax.legend(loc='lower center', bbox_to_anchor=(.5, 1.0), ncol=4)

ax2.plot(L1, L3, color='darkorange', linewidth=1)
ax2.set_ylabel('Probability')
ax2.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1.0])
ax2.set_ylim(-0.045, 1)

plt.show()